In [1]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import open3d as o3d
import torch
from skimage import measure
from tqdm import tqdm, trange

from core.dataset import ScanNet
from core.integrate import PanopticFusionScalableTSDFVolume
from core.integrate.utils.misc import instance_id_to_one_hot_mask

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
tsdf_device = "cuda:1"
tsdf_volume = PanopticFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device=tsdf_device,
)

save_dir = "/home/quanta/Experiments/feature-instance-fusion/scannet_scene0000_00/"
tsdf_volume.load(save_dir + "tsdf/tsdf_vol.pt")

In [4]:
dataset = ScanNet("/home/quanta/Datasets/ScanNet/")
scan_id = "scene0000_00"
id = dataset.scan_id_list.index(scan_id)
inst = dataset[id]
single_instance = inst["scan_dataset"]
H, W = inst["color_height"], inst["color_width"]

: 

In [5]:
tsdf_volume.reset_instance()

In [18]:
inputs = single_instance.get_torch_tensor(
    0,
    device=tsdf_device,
    keys={
        "label"
    },
)
torch.nn.functional.one_hot(inputs['label'].long(), num_classes=41).shape
# inputs['label'].shape

torch.Size([968, 1296, 41])

In [35]:
indent = 1
for idx in trange(0, len(single_instance), indent):
    # for idx in trange(100):
    inputs = single_instance.get_torch_tensor(
        idx,
        device=tsdf_device,
        keys={
            "depth",
            "depth_intr",
            "pose",
            "instance",
            "color_intr",
        },
    )
    masks = instance_id_to_one_hot_mask(instance=inputs["instance"], background_id=0)

    result = tsdf_volume.integrate_instance_with_existing_voxel(
        masks=masks,
        masks_intr=inputs["color_intr"],
        cam_pose=inputs["pose"],
        threshold=0.25,  # 0.25 for gt, 0.15 for grounding sam
        depth_type="voxel_rc",
        # depth_type="mesh_rc",
        # depth_type="sensor",
        # depth_type="voxel_rc_torch",
        # depth=inputs['depth'],
        # depth_intr=inputs['depth_intr'],
    )
    # print(result)

100%|██████████| 5578/5578 [04:52<00:00, 19.07it/s]


In [7]:
# mesh_rc 20.46 it/s
# sensor 47.59 it/s
# voxel_rc 21.12 it/s
# voxel_rc_torch 1.58 it/s

In [8]:
import open3d as o3d

draw = o3d.visualization.EV.draw

In [36]:
verts, faces = tsdf_volume.extract_mesh()
mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(verts),
    triangles=o3d.utility.Vector3iVector(faces),
)
labels, labels_w_sum = tsdf_volume.extract_label_on_grid(verts)

In [22]:
color_set = np.random.random(size=(tsdf_volume._instance_label_num, 3))

In [37]:
color = color_set[labels]
mesh.vertex_colors = o3d.utility.Vector3dVector(color)
mesh.compute_vertex_normals()

TriangleMesh with 1120162 points and 2091565 triangles.

In [ ]:
draw([mesh])